In [2]:
pip install oracledb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 28.2 MB/s eta 0:00:00


In [3]:
import oracledb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import config
import scipy.stats as sps


warnings.filterwarnings("ignore", category=UserWarning)

connection = oracledb.connect(user=config.user, password=config.password,
                              dsn=config.dsn)
cursor = connection.cursor()

In [19]:
df_perf = pd.read_sql('select * from performances', connection)
df_param = pd.read_sql('select * from parameters', connection)
df_sim = pd.read_sql('select * from simulations', connection)

In [158]:
df_param['E_GREEDY'] = df_param['E_GREEDY'].fillna(0)
df_param['C'] = df_param['C'].fillna(0)
df = pd.merge(df_param, df_perf, on="PARAMETER_ID")
df = pd.merge(df, df_sim, on="SIM_ID")
df['AGR_COEFF'] = df.apply(lambda x: x['ATK_CNT'] / x['MOVE_CNT'], axis=1)
df['OUTCOME'] = df.apply(lambda x: 'win' if x['PARAMETER_ID']==x['OUTCOME'] else 'draw' if x['OUTCOME']=='draw' else 'lose', axis=1)
df_mcts = df[df['ALGORITHM_ID']=='MCTS']
df_dlmc = df[df['ALGORITHM_ID']=='DLMC']
df_mcts = df_mcts.drop(['PARAMETER_ID','PERFORMANCE_ID','ALGORITHM_ID', 'PLAYOUT_NUMBER', 'SIM_ID', 'RED_PLAYER', 'BLUE_PLAYER', 'TURN_TIME'], axis= 1)
df_dlmc = df_dlmc.drop(['PARAMETER_ID','PERFORMANCE_ID','ALGORITHM_ID', 'SIM_ID', 'OUTCOME', 'RED_PLAYER', 'BLUE_PLAYER', 'TURN_TIME'], axis= 1)
df_mcts_dl = df_mcts[df_mcts['SIM_DEPTH'].isna() == False]
df_mcts_ndl = df_mcts[df_mcts['SIM_DEPTH'].isna()].drop(['SIM_DEPTH', 'OUTCOME'], axis=1)
print(f'dlmc: {len(df_dlmc.index)}\nmcts_dl: {len(df_mcts_dl.index)}\nmcts_ndl: {len(df_mcts_ndl.index)}')

dlmc: 3592
mcts_dl: 4702
mcts_ndl: 11006


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures

In [156]:
X = df_dlmc['C'].values
y = df_dlmc['AGR_COEFF']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)
model = LinearRegression()
model.fit(X_train.reshape(-1, 1),y_train)
predictions = model.predict(X_test.reshape(-1, 1))

print('mean_squared_error : ', mean_squared_error(y_test, predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))
# model.predict([[0.8]])

mean_squared_error :  0.047130620435496345
mean_absolute_error :  0.1542325947587164


In [157]:
X = df_mcts_ndl.drop(['TURN_CNT'],axis= 1)
y = df_mcts_ndl['TURN_CNT']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)
model = LinearRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

print('mean_squared_error : ', mean_squared_error(y_test, predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))
# model.predict([X_test.iloc[1]])

mean_squared_error :  12.187928328641453
mean_absolute_error :  2.7590063090947523


In [133]:
X = df_mcts_dl.drop(['OUTCOME'],axis= 1)
y = df_mcts_dl['OUTCOME']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)
model = LogisticRegression()
model.fit(X_train,y_train)
f'acc: {model.score(X_test, y_test)}'
# model.predict([X_test.iloc[0]])

'acc: 0.9355067328136074'

In [146]:
X = df_dlmc['C'].values
y = df_dlmc['AGR_COEFF']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)
pr = PolynomialFeatures(degree = 4)
lr_2 = LinearRegression()
lr_2.fit(pr.fit_transform(X_train.reshape(-1, 1)), y_train)
predictions = lr_2.predict(pr.fit_transform(X_test.reshape(-1, 1)))

print('mean_squared_error : ', mean_squared_error(y_test, predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, predictions))
# lr_2.predict(pr.fit_transform([[0.8]]))

mean_squared_error :  0.04716492278724634
mean_absolute_error :  0.1541866528406332
